**Drive Permission to access dataset from google drive.**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Relative location of drive to DIP project folder
drive_loc='/content/gdrive/My Drive/DIP/'

In [2]:
# Importing required python packages!
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from PIL import Image

Using TensorFlow backend.


# Dataset

In [0]:
# Extracting dataset
import zipfile
path_to_zip_file = drive_loc + 'digit-recognizer.zip'
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
zip_ref.extractall()
zip_ref.close()

**Storing Images for Vizualization**

In [4]:
# Creating Images directory
!rm -r images
!mkdir images 

rm: cannot remove 'images': No such file or directory


In [0]:
Images = np.zeros((28, 28))
c = 0
with open("train.csv") as f:
  for i,line in enumerate(f):
    if c > 0:
      X = line[2:].split(',')
      for j in range(len(X)):
        X[j] = int(X[j])
      for p in range(28):
        Images[p,:] = X[p*28:(p+1)*28]
      if c < 100:
        im = Image.fromarray(Images)
        im = im.convert('RGB')
        im.save('images/'+str(c)+".png")
      else:
        break
    c += 1

**Parameters of training**

In [0]:
batch_size = 128
num_classes = 10
epochs = 10

In [0]:
img_rows, img_cols = 28, 28

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)      

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

# Training

In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 129us/step - loss: 0.2611 - acc: 0.9201 - val_loss: 0.0578 - val_acc: 0.9823
Epoch 2/10
60000/60000 [==============================] - 5s 79us/step - loss: 0.0885 - acc: 0.9736 - val_loss: 0.0423 - val_acc: 0.9863
Epoch 3/10
60000/60000 [==============================] - 5s 76us/step - loss: 0.0655 - acc: 0.9808 - val_loss: 0.0361 - val_acc: 0.9871
Epoch 4/10
60000/60000 [==============================] - 5s 75us/step - loss: 0.0541 - acc: 0.9835 - val_loss: 0.0333 - val_acc: 0.9883
Epoch 5/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.0456 - acc: 0.9864 - val_loss: 0.0289 - val_acc: 0.9901
Epoch 6/10
60000/60000 [==============================] - 4s 75us/step - loss: 0.0416 - acc: 0.9879 - val_loss: 0.0281 - val_acc: 0.9905
Epoch 7/10
60000/60000 [==============================] - 4s 75us/step - loss: 

# Testing

In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 60us/step
Test loss: 0.030512718466916885
Test accuracy: 0.99
